In [65]:
import math
import time
import pyaudio
import numpy as np
from threading import Event, Lock, Thread

PyAudio = pyaudio.PyAudio

In [16]:
bitrate = 16000
frequency = 500
seconds = 5
frequencyChangeAccuracySeconds = 1

scale = bitrate / math.pi
packetSize = int(bitrate * frequencyChangeAccuracySeconds)

data = []
while len(data) < packetSize:
    data.append('')

p = PyAudio()
stream = p.open(format = p.get_format_from_width(1), 
                channels = 1, 
                rate = bitrate,
                output = True)

frame = 0
numberOfFrames = int(bitrate * seconds)
startTime = None
while frame < numberOfFrames:
    packetFrameLimit = min(frame + packetSize, numberOfFrames)
    framesToWrite = packetFrameLimit - frame
    
    i = 0
    while frame < packetFrameLimit:
        x = frequency * frame / scale
        data[i] = chr(int(math.sin(x))*127+128)
        i = i + 1
        frame = frame + 1
        
    stream.write(''.join(data), num_frames=framesToWrite)
    
    if startTime is None:
        startTime = time.time()
        
    actualSecondsElapsed = time.time() - startTime
    targetSecondsElapsed = frame / float(bitrate)
    
    delta = targetSecondsElapsed - actualSecondsElapsed
    if delta > 0:
        time.sleep(delta)
        
    lastTime = time.time()
   
stream.stop_stream()
stream.close()
p.terminate()

In [72]:
bitrate = 8000
frequency = 500

playerReadyToClose = Event()
playerReadyToClose.clear()

def player():
    p = PyAudio()
    stream = p.open(format = pyaudio.paFloat32, 
                    channels = 1, 
                    rate = bitrate,
                    output = True)

    frame = 0
    frameSeconds = 1.0 / bitrate
    scale = bitrate / np.pi / 2.0
    deltaAverage = 0
    lastTime = time.time()
    data = np.array([ 0 ]).astype(np.float32)
    while not playerReadyToClose.is_set():
        x = frequency * frame / scale
        data[0] = math.sin(x)
        stream.write(data.tobytes())
        newTime = time.time()
        deltaAverage = (deltaAverage * frame + (frameSeconds - (newTime - lastTime))) / (frame + 1)
        if deltaAverage > 0:
            time.sleep(deltaAverage)
        lastTime = newTime
        frame = frame + 1

    stream.stop_stream()
    stream.close()
    p.terminate()

playerThread = Thread(target = player,
                      name="player",
                      args=[])
playerThread.start()

In [88]:
playerReadyToClose.set()
np.std([ 1, 2, 3 ])

0.816496580927726